### Import librairies :

In [ ]:
import pandas as pd
import numpy as np

### Import des TSV : 

**PERSONNALITES**

In [ ]:
name_DF = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep = "\t", low_memory = False)
name_DF.head()

KernelInterrupted: Execution interrupted by the Jupyter kernel.

**FILMS SELON LES PAYS**

In [ ]:
akas_DF = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep = "\t", low_memory = False)
akas_DF.head(50)

**FILMS : GENRES, ANNEES DE PRODUCTION...**

In [ ]:
basics_DF = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = "\t", low_memory = False)
basics_DF.head()

**DIRECTEURS**

In [ ]:
crew_DF = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep = "\t", low_memory = False)
crew_DF.head()

**ZOOM SUR LES SERIES**

In [ ]:
episode_DF = pd.read_csv('https://datasets.imdbws.com/title.episode.tsv.gz', sep = "\t", low_memory = False)
episode_DF.head()

**CASTING PAR FILMS YC EQUIPE DE REALISATION**

In [ ]:
principals_DF = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep = "\t", low_memory = False)
principals_DF.head()

**RATINGS**

In [ ]:
ratings_DF = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep = "\t", low_memory = False)
ratings_DF.head()

### DF Personnalités : 

In [ ]:
# Liste des personnalités nettoyée des NaN dans colonne 'primaryProfession': 
# Utilisation de .dropna() :

List_Act_Clean_DF = name_DF.dropna(subset = ['primaryProfession', 'knownForTitles'])
print(List_Act_Clean_DF)

In [ ]:
# Liste nettoyée des \N dans colonnes 'primaryProfession', 'birthYear et 'knowForTitles' : 

List_Act_Clean_DF2 = List_Act_Clean_DF[List_Act_Clean_DF['primaryProfession'] != '\\N']
List_Act_Clean_DF3 = List_Act_Clean_DF2[List_Act_Clean_DF2['birthYear'] != '\\N']
List_Act_Clean_DF4 = List_Act_Clean_DF3[List_Act_Clean_DF3['knownForTitles'] != '\\N']
print(List_Act_Clean_DF4)

**Liste Acteurs vs Productions tout types confondus :**

In [ ]:
# 1) Liste des acteurs/Actrices - utilisation de .str.contains('x|y') :

List_Act_Clean_DF5 = List_Act_Clean_DF4[List_Act_Clean_DF4['primaryProfession'].str.contains('actor|actress')]
print(List_Act_Clean_DF5)

In [ ]:
# Fractionnement colonne 'knownForTitles' pour avoir un film par acteur/actrice et par ligne : 
# Etape 1 : utilisation de .set_index() pour mettre en index toutes les colonnes qui ne doivent pas être fractionnées :

List_Act_Clean_DF6 = List_Act_Clean_DF5.set_index(['nconst', 'primaryName', 'birthYear', 'deathYear','primaryProfession'])
print(List_Act_Clean_DF6)

In [ ]:
# Fractionnement colonne 'knownForTitles' pour avoir un film par acteur/actrice et par ligne : 
# Etape 2 : utilisation de .stack() pour pivoter l'en-tête de la colonne 'knownForTitles' :

List_Act_Clean_DF7 = List_Act_Clean_DF6.stack()
print(List_Act_Clean_DF7)

In [ ]:
# Fractionnement colonne 'knownForTitles' pour avoir un film par acteur/actrice et par ligne : 
# Etape 3 : utilisation de .str.split('', expand = True) pour spliter les Id des films selon leur séparateur et mettre 1 iD par ligne (si expand = False, split des Id mais en colonnes)

List_Act_Clean_DF8 = List_Act_Clean_DF7.str.split(',', expand=True)
print(List_Act_Clean_DF8)

In [ ]:
# Fractionnement colonne 'knownForTitles' pour avoir un film par acteur/actrice et par ligne : 
# Etape 4 : utilisation de .stack() pour regrouper les films par acteurs/actrices et leur mettre un 'index' avant :

List_Act_Clean_DF9 = List_Act_Clean_DF8.stack()
print(List_Act_Clean_DF9)

In [ ]:
# Fractionnement colonne 'knownForTitles' pour avoir un film par acteur/actrice et par ligne : 
# Etape 5 : utilisation de .unstack(-2) pour dépivoter le nom de la colonne 'knownForTitles' :

List_Act_Clean_DF10 = List_Act_Clean_DF9.unstack(-2)
print(List_Act_Clean_DF10)

In [ ]:
# Fractionnement colonne 'knownForTitles' pour avoir un film par acteur/actrice et par ligne : 
# Etape 6 : utilisation de .reset_index(-1, drop = True) pour supprimer l'index temporaire qui est dans l'avant dernière colonne :


List_Act_Clean_DF11 = List_Act_Clean_DF10.reset_index(-1, drop=True)
print(List_Act_Clean_DF11)

In [ ]:
# Fractionnement colonne 'knownForTitles' pour avoir un film par acteur/actrice et par ligne : 
# Etape 7 : utilisation de .reset_index() pour rebasculer les colonnes indexées en étape 1 en colonnes normales :

List_Act_Clean_DF12 = List_Act_Clean_DF11.reset_index()
print(List_Act_Clean_DF12)

**Liaison Acteurs vs Films/Série etc... :**

In [ ]:
# SELECT t1.*, t2.*
# FROM df_1 t1
# INNER JOIN df_2 t2
# ON t1.index = t2.index


Product_vs_act_DF = pd.merge(List_Act_Clean_DF12,basics_DF, how = 'inner', left_on=['knownForTitles'], right_on=['tconst'])
print(Product_vs_act_DF)

**Moyenne âge acteurs de films/série etc... :**

In [ ]:
# Etape 1 : Sélection des lignes où les films ont des dates de production (discimination des films n'en n'ayant pas): 

Product_vs_act_DF = Product_vs_act_DF[Product_vs_act_DF['startYear'] != '\\N']
print(Product_vs_act_DF)

In [ ]:
# Etape 2 : transformation des colonnes 'startYear' et 'birthYear' en int : 

Product_vs_act_DF['startYear'] = Product_vs_act_DF['startYear'].astype(int)
Product_vs_act_DF['birthYear'] = Product_vs_act_DF['birthYear'].astype(int)



# Etape 3 : ajout de la colonne 'Age' correspondant à l'âge de l'acteur/actrice au moment de la production du film : 

Product_vs_act_DF['Age'] = Product_vs_act_DF['startYear'] - Product_vs_act_DF['birthYear']
print(Product_vs_act_DF)

In [ ]:
# Etape 4 : calcul de la moyenne d'âge avec .mean() + utilisation de round() pour arrondir le résultat : 

Moy_Age_Act = round(Product_vs_act_DF['Age'].mean())
print(Moy_Age_Act)

**Acteurs les + présents, quel que soit le type de production :**

**--> Utilisation du DF liaisonné Product_vs_act_DF :**

In [ ]:
Act_most_present_DF = Product_vs_act_DF['primaryName'].value_counts(sort = True, ascending = False).head(5)
print('Les 5 acteurs/actrices les plus présents quel que soit le type de production (films, séries..) sont : \n',Act_most_present_DF)

**Acteurs les + présents, quel que soit le type de production, selon les périodes :**

**--> Utilisation du DF liaisonné Product_vs_act_DF :**

In [ ]:
# Etape 1 : Définition des périodes (=décennies) via ajout colonne :

Product_vs_act_DF['Période'] = (Product_vs_act_DF['startYear'] //10) *10
print(Product_vs_act_DF)

In [ ]:
# Etape 2 : calcul fréquence selon période :

Act_most_present_by_Decades_DF = Product_vs_act_DF.groupby(['Période','primaryName']).size()
Act_most_present_by_Decades_DF

**Acteurs/actrices vs Movies_P1960_DF :**


In [ ]:
# Filtres : 

# Products après 1960 : 

Product_P1960_vs_act_DF = Product_vs_act_DF[Product_vs_act_DF['startYear']>= 1960]

# Films et films TV après 1960 : 

Films__P1960_vs_act_DF = Product_P1960_vs_act_DF[Product_P1960_vs_act_DF['titleType'].str.contains('Movie|movie')] 
print(Films__P1960_vs_act_DF)

**Moyenne âge acteurs/actrices de films/films tv produits après 1960 (DataSet) :**

In [ ]:
Moy_Age_Act_Films_P1960 = round(Films__P1960_vs_act_DF['Age'].mean())
print(Moy_Age_Act_Films_P1960)

**Acteurs/actrices de films/films tv produits après 1960 (DataSet) les + présents :**

In [ ]:
Act_most_present_FilmsP1960_DF = Films__P1960_vs_act_DF['primaryName'].value_counts(sort = True, ascending = False).head(5)
print('Les 5 acteurs/actrices les plus présents quel que soit le type de production (films, séries..) sont : \n',Act_most_present_FilmsP1960_DF)

**Acteurs/actrices de films/films tv produits après 1960 (DataSet) les + présents selon décennies :**

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bb4ebdbb-68fe-4d65-bdae-b774c78fb714' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>